In [1]:
# Celda 1: Importaciones, Carga de Datos y Configuración Inicial
import os
import re
import pandas as pd
import faiss
import psutil
from dotenv import load_dotenv
from tqdm.auto import tqdm
from joblib import Memory
import gradio as gr
from IPython.display import display # Importado para display(df.head())

# LlamaIndex Core y componentes
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings,
    Document
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.prompts import PromptTemplate
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter
from llama_index.llms.google_genai import GoogleGenAI

# --- 1. Configurar Entorno ---
num_cores = psutil.cpu_count(logical=True)
os.environ['OMP_NUM_THREADS'] = str(num_cores)
os.environ['MKL_NUM_THREADS'] = str(num_cores)
print(f"Librerías importadas. ¡Entorno listo con {num_cores} núcleos disponibles!")

# --- 2. Definir Rutas y Cargar Metadatos ---
METADATA_FILE = os.path.join("data", "metadatos_chatbot_final.csv")

def load_metadata_dataframe(filepath):
    """Carga los metadatos desde un CSV a un DataFrame de Pandas."""
    try:
        df = pd.read_csv(filepath)
        # Llenamos los valores vacíos con "No especificado" para evitar errores
        df.fillna("No especificado", inplace=True)
        print("Metadatos cargados y procesados.")
        display(df.head())
        return df
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo de metadatos en la ruta: {filepath}")
        return None
    except Exception as e:
        print(f"ERROR al cargar metadatos: {e}.")
        return None

# Cargar el DataFrame al inicio
df_metadatos = load_metadata_dataframe(METADATA_FILE)

Librerías importadas. ¡Entorno listo con 6 núcleos disponibles!
Metadatos cargados y procesados.


,Año de publicación,Código sugerido,Nombre del archivo,Título del estudio,Categoría,Tipo de documento,Subcategoría,Idioma,Número de páginas,Incorpora perspectiva de género,...,Entidad solicitante,Entidad a cargo del estudio,Investigador u organismo principal,Equipo de investigación,Número de documento administrativo,Tipo de financiamiento,Costo del estudio,Base de datos enviada,Base pública,Url
0,1999,E99-0000,Mercado Laboral en Docencia,Análisis del Mercado de Servicios de Docencia ...,Estudios,Informe,Docentes,Español,123,No,...,Ministerio de Educación de Chile; MINEDUC,Facultad de Ciencias Económicas y Administrati...,"Bravo, David","Ruiz-Tagle, Jaime; Sanhueza, Ricardo",Sin información,Interno,0,No,No,https://bibliotecadigital.mineduc.cl/bitstream...
1,2000,E00-0001,Evaluación Resultados ENLACES,Diseño de un modelo de evaluación de resultado...,Estudios,Informe,TIC,Español,53,No,...,Enlaces; Ministerio de Educación de Chile; MIN...,Asesorías para el Desarrollo,"Raczynski, Dagmar","Pavez, M. Angélica",Sin información,Interno,0,No,No,https://bibliotecadigital.mineduc.cl/bitstream...
2,2001,E01-0001,Focalización Becas Liceos Para Todos CC,Focalización de Becas del Programa Liceo para ...,Estudios,Informe,Monitoreo y evaluación,Español,30,No,...,Ministerio de Educación de Chile; MINEDUC,"Departamento de Salud Pública, PUC","Marshall, Guillermo","Correa, Lorena",Sin información,Interno,0,No,No,https://bibliotecadigital.mineduc.cl/bitstream...
3,2001,E01-0002,Focalización Becas Liceos Para Todos ECO,Focalización de Becas del Programa Liceo para ...,Estudios,Informe,Monitoreo y evaluación,Español,61,No,...,Ministerio de Educación de Chile; MINEDUC,"Departamento de Salud Pública, PUC","Marshall, Guillermo","Correa, Lorena",Sin información,Interno,0,No,No,https://bibliotecadigital.mineduc.cl/bitstream...
4,2001,E01-0003,OFT Educación Media,Objetivos Fundamentales Transversales en la En...,Estudios,Informe,Currículo,Español,61,No,...,Ministerio de Educación de Chile; MINEDUC,"UCE, MINEDUC","Fernández, Carolina","Jashes, Jessana",Sin información,Interno,0,No,No,https://bibliotecadigital.mineduc.cl/bitstream...


In [2]:
# Celda 2: CONFIGURACIÓN OPTIMIZADA DE MODELOS DE IA

# Cargar la clave de API desde el archivo .env
load_dotenv()
if "OPENROUTER_API_KEY" in os.environ:
    print("Clave de API de OpenRouter cargada.")
else:
    print("¡ADVERTENCIA! No se encontró la clave de API de OpenRouter.")

if "HF_TOKEN" in os.environ:
    print("Token de Hugging Face (HF_TOKEN) encontrado en el entorno.")
else:
    print("¡ADVERTENCIA! No se encontró el token de Hugging Face.")

# --- Configuración global de LlamaIndex optimizada para CPU ---
optimal_batch_size = max(64, num_cores * 8)
print(f"Configurando el modelo de embeddings con batch size óptimo de {optimal_batch_size} para {num_cores} núcleos.")

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
    embed_batch_size=optimal_batch_size,
    cache_folder="./model_cache",
    normalize=True,
    trust_remote_code=True
)

# Selecciona el proveedor de LLM aquí: "openrouter" o "gemini"
LLM_PROVIDER = "gemini"

if LLM_PROVIDER == "gemini":
    Settings.llm = GoogleGenAI(
        api_key=os.getenv("GOOGLE_API_KEY"),
        model="gemini-1.5-flash-latest",
        temperature=0.1,
    )
    print("Usando Gemini como modelo LLM.")
else:
    Settings.llm = OpenRouter(
        model="google/gemma-3n-e4b-it:free",
        #model="google/gemma-3n-e2b-it:free",
        #model="mistralai/mistral-small-3.2-24b-instruct:free",
        #model="deepseek/deepseek-chat-v3-0324:free",
        #model="deepseek/deepseek-r1-0528:free",
        #model="deepseek/deepseek-chat:free",
        #model="google/gemini-2.0-flash-exp:free",
        #model="mistralai/mistral-nemo:free",
        #model="qwen/qwq-32b:free",
        #model="microsoft/mai-ds-r1:free",
        #model="meta-llama/llama-4-maverick:free",
        temperature=0.1, 
    )
    print("Usando OpenRouter como modelo LLM.")

# Función para monitorear uso de memoria
def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Uso de memoria actual: {memory_info.rss / (1024 * 1024):.2f} MB")

print("\nConfiguración de modelos completada.")
print_memory_usage()

Clave de API de OpenRouter cargada.
Token de Hugging Face (HF_TOKEN) encontrado en el entorno.
Configurando el modelo de embeddings con batch size óptimo de 64 para 6 núcleos.
Usando Gemini como modelo LLM.

Configuración de modelos completada.
Uso de memoria actual: 553.79 MB


In [5]:
# Celda 3: Creación o Carga del Índice Vectorial (con Caching)

# 1. Configuramos la ubicación de la caché de joblib
CACHE_DIR = "./joblib_cache"
memory = Memory(CACHE_DIR, verbose=1)

# 2. Función para crear el índice (se ejecutará solo si no está en caché)
@memory.cache
def get_or_create_index(dataframe):
    print("="*50)
    print("INICIANDO PROCESO DE CREACIÓN DE ÍNDICE (LENTO LA PRIMERA VEZ)...")
    print("="*50)
    
    if dataframe is None:
        print("No se puede crear el índice porque el DataFrame es nulo.")
        return None

    Settings.node_parser = SentenceSplitter(chunk_size=2048)

    def optimize_text_for_embedding(text, max_length=1500):
        if not isinstance(text, str): return ""
        text = text[:max_length]
        text = re.sub(r'\\s+', ' ', text)
        return text.strip()
        
    print("Convirtiendo metadatos en documentos de LlamaIndex...")
    metadata_documents = []
    columnas_a_incluir = [
        'Año de publicación', 'Código sugerido', 'Nombre del archivo', 'Título del estudio', 
        'Categoría', 'Tipo de documento', 'Subcategoría', 'Idioma', 'Número de páginas', 
        'Incorpora perspectiva de género', 'Año de término', 'Lugar de término', 
        'Palabras clave', 'Objetivo', 'Metodología', 'Resumen', 'Documento público', 
        'Publicación destacada', 'Publicado', 'Editorial', 'Entidad solicitante',
        'Entidad a cargo del estudio', 'Investigador u organismo principal', 'Equipo de investigación', 
        'Número de documento administrativo', 'Tipo de financiamiento', 'Costo del estudio', 
        'Base de datos enviada', 'Base pública', 'Url'
    ]
    columnas_prioritarias = ['Título del estudio', 'Resumen', 'Palabras clave', 'Objetivo', 
                             'Metodología', 'Investigador u organismo principal']

    for index, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0], desc="Procesando metadatos"):
        partes_texto = []
        for col in columnas_prioritarias:
            if col in row and pd.notna(row[col]) and row[col] != "No especificado":
                texto_optimizado = optimize_text_for_embedding(str(row[col]), 300)
                if texto_optimizado:
                    partes_texto.append(f"{col}: {texto_optimizado}"); partes_texto.append(f"{col}: {texto_optimizado}")
        
        for col in columnas_a_incluir:
            if col not in columnas_prioritarias:
                if col in row and pd.notna(row[col]) and row[col] != "No especificado":
                    texto_optimizado = optimize_text_for_embedding(str(row[col]), 200)
                    if texto_optimizado:
                        partes_texto.append(f"{col}: {texto_optimizado}")
        
        contenido_buscable = ". ".join(partes_texto)
        doc = Document(
            text=contenido_buscable,
            metadata={col: optimize_text_for_embedding(str(row.get(col, '')), 500) for col in columnas_a_incluir}
        )
        metadata_documents.append(doc)
    
    print("Creando índice vectorial con FAISS...")
    d = len(Settings.embed_model.get_text_embedding("test"))
    faiss_index = faiss.IndexFlatL2(d)
    vector_store = FaissVectorStore(faiss_index=faiss_index)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    index = VectorStoreIndex.from_documents(
        metadata_documents,
        storage_context=storage_context,
        show_progress=True 
    )
    
    print("="*50)
    print("¡ÍNDICE CREADO Y GUARDADO EN CACHÉ EXITOSAMENTE!")
    print("="*50)
    
    return index

# 3. Llamamos a la función, pasándole el dataframe
if 'df_metadatos' in locals() and df_metadatos is not None:
    metadata_index = get_or_create_index(df_metadatos)
else:
    print("No se puede crear el índice porque el DataFrame de metadatos no se cargó. Revisa la Celda 1.")
    metadata_index = None

In [ ]:
# Celda 4: Lógica del Chatbot e Interfaz Gradio

if 'metadata_index' not in locals() or metadata_index is None or 'df_metadatos' not in locals() or df_metadatos is None:
    print("ERROR: El índice o el DataFrame no han sido creados. Por favor, ejecuta las celdas anteriores correctamente.")
else:
    # --- Pre-cálculo de autores conocidos ---
    autores_principales = set(df_metadatos['Investigador u organismo principal'].str.lower().unique())
    autores_equipo_raw = set(df_metadatos['Equipo de investigación'].str.lower().unique())
    autores_equipo = set()
    for item in autores_equipo_raw:
        if isinstance(item, str):
            nombres = re.split(r'[;,]', item)
            for nombre in nombres:
                if nombre.strip():
                    autores_equipo.add(nombre.strip())
    KNOWN_AUTHORS = autores_principales.union(autores_equipo)
    print(f"Se identificaron {len(KNOWN_AUTHORS)} autores únicos para la búsqueda precisa.")

    # --- GESTOR DE ESTADO DE LA CONVERSACIÓN ---
    chat_state = {"last_query_term": None, "last_results": [], "last_author_filter": None, "current_page": 0}

    # --- Funciones de apoyo ---
    def get_relevance_label(score):
        if score >= 0.75: return "Muy Alta"
        elif score >= 0.5: return "Alta"
        elif score >= 0.25: return "Media"
        else: return "Baja"
    
    def find_relevance_reason(node_text, search_terms_str):
        search_terms = [term.strip() for term in search_terms_str.split(',') if term.strip()]
        reasons = []
        for term in search_terms:
            match = re.search(re.escape(term), node_text, re.IGNORECASE)
            if match:
                start, end = match.span()
                context_start = max(0, start - 50)
                context_end = min(len(node_text), end + 70)
                context_snippet = node_text[context_start:context_end].strip().replace(match.group(0), f"**{match.group(0)}**")
                text_before_match = node_text[:start]
                metadata_match = re.findall(r"(\b[\w\s'óáéíúñü]+\b):\s", text_before_match)
                metadata_name = metadata_match[-1] if metadata_match else "Contenido"
                reasons.append(f"Término '{term}' encontrado en **{metadata_name}** (contexto: *'...{context_snippet}...'*)\n")
        if not reasons: return f"Relevancia semántica general con '{search_terms_str}'"
        return "".join(reasons)

    def search_by_author(author_name):
        print(f"Búsqueda precisa por autor: '{author_name}'.")
        mask = (df_metadatos['Investigador u organismo principal'].str.contains(author_name, case=False, na=False, regex=False) |
                df_metadatos['Equipo de investigación'].str.contains(author_name, case=False, na=False, regex=False))
        author_hits_df = df_metadatos[mask].copy()
        author_hits_df['Año de publicación num'] = pd.to_numeric(author_hits_df['Año de publicación'], errors='coerce').fillna(0)
        sorted_hits_df = author_hits_df.sort_values(by='Año de publicación num', ascending=False)
        num_docs_encontrados = len(sorted_hits_df)
        if num_docs_encontrados == 0: return f"No he encontrado documentos del autor '{author_name}'. Prueba con otro nombre o solo el apellido."
        header = f"He encontrado {num_docs_encontrados} documento(s) del autor '{author_name}', ordenados por año (del más reciente al más antiguo):\n\n"
        lista_formateada = []
        for i, (index, row) in enumerate(sorted_hits_df.iterrows(), 1):
            titulo, autores, ano_str, url = row.get('Título del estudio', 'N/A'), f"{row.get('Investigador u organismo principal', 'N/A')}, {row.get('Equipo de investigación', 'N/A')}", str(row.get('Año de publicación', 'N/A')), row.get('Url', 'No disponible')
            try: ano = str(int(float(ano_str)))
            except (ValueError, TypeError): ano = ano_str
            info_doc = (f"{i}. **Título:** {titulo}\n   - **Autor(es):** {autores}\n   - **Año:** {ano}\n   - **URL:** {url}")
            lista_formateada.append(info_doc)
        return header + "\n\n".join(lista_formateada)

    def format_results_page():
        total_found, current_page, llm_term = len(chat_state["last_results"]), chat_state["current_page"], chat_state["last_query_term"]
        if total_found == 0: return f"No he encontrado documentos relevantes sobre '{llm_term}'. Por favor, intenta reformular tu búsqueda."
        MAX_TO_SHOW, start_index = 5, (current_page - 1) * 5
        nodes_to_show = chat_state["last_results"][start_index:start_index + MAX_TO_SHOW]
        if not nodes_to_show: return f"No hay más documentos que mostrar sobre '{llm_term}'. Ya estás en la última página."
        header = ""
        if current_page == 1:
            header = f"He encontrado un total de {total_found} documentos relevantes sobre '{llm_term}'. Mostrando los {len(nodes_to_show)} más relevantes (página {current_page}):\n\n"
        else:
            header = f"Mostrando la página {current_page} de {-(-total_found // MAX_TO_SHOW)} para '{llm_term}' (documentos {start_index + 1}-{start_index + len(nodes_to_show)}):\n\n"
        
        lista_formateada = []
        for i, res in enumerate(nodes_to_show, start=start_index + 1):
            metadata, score, reason_text = res['node'].metadata, res['score'], res['reason']
            relevance_label, relevance_percentage, url = get_relevance_label(score), f"{score:.0%}", metadata.get('Url', 'No disponible')
            info_doc = (f"{i}. **Título:** {metadata.get('Título del estudio', 'N/A')} **(Relevancia: {relevance_label} [{relevance_percentage}])**\n"
                        f"   - **Autor(es):** {metadata.get('Investigador u organismo principal', 'N/A')}, {metadata.get('Equipo de investigación', 'N/A')}\n"
                        f"   - **Año:** {metadata.get('Año de publicación', 'N/A')}\n"
                        f"   - **URL:** {url}\n"
                        f"   - ***Motivo:*** *{reason_text}*")
            lista_formateada.append(info_doc)
        
        footer = "\n\n--- \n*Para ver más, escribe 'siguiente', 'más' o 'página {current_page + 1}'.*" if (start_index + MAX_TO_SHOW) < total_found else ""
        return header + "\n\n".join(lista_formateada) + footer

    def search_by_topic(message):
        print(f"Búsqueda semántica por tema: '{message}'.")
        print("Extrayendo concepto clave con LLM...")
        extraction_template = PromptTemplate(
            "Eres un motor de búsqueda semántica. Reescribe la pregunta del usuario en una consulta optimizada (una frase o conceptos clave separados por comas) para encontrar documentos en una base de datos académica.\n"
            "Ejemplo 1: 'estudios sobre brecha de genero' -> brecha de género, desigualdad de género en educación\n"
            "Ejemplo 2: 'documentos sobre evaluacion docente' -> evaluación docente, desempeño de profesores\n"
            "---\nPregunta original: '{query_str}'\nConsulta optimizada:"
        )
        prompt_final = extraction_template.format(query_str=message)
        response = Settings.llm.complete(prompt_final)
        llm_term = str(response).strip().lower()
        print(f"Término de búsqueda final: '{llm_term}'")
        
        retriever = metadata_index.as_retriever(similarity_top_k=30)
        retrieved_nodes = retriever.retrieve(llm_term)
        
        relevant_results = []
        RELEVANCE_THRESHOLD = 0.40
        if retrieved_nodes:
            all_potential_results = []
            for node_with_score in retrieved_nodes:
                detailed_reason = find_relevance_reason(node_with_score.node.text, llm_term)
                all_potential_results.append({"node": node_with_score.node, "score": node_with_score.score, "reason": detailed_reason})
            
            all_potential_results.sort(key=lambda x: x['score'], reverse=True)
            for result in all_potential_results:
                if result["score"] >= RELEVANCE_THRESHOLD:
                    relevant_results.append(result)
                    
        chat_state.update({"last_query_term": llm_term, "last_results": relevant_results, "current_page": 1})
        return format_results_page()

    # --- ROUTER DE IA Y GESTORES DE INTENCIONES ---
    def get_user_intent(user_message):
        intent_classifier_prompt = PromptTemplate(
            "Clasifica la intención del usuario en una categoría: 'saludo', 'ayuda', 'chitchat', 'paginacion', 'busqueda_por_autor', 'busqueda_por_tema'.\n"
            "Ejemplos:\n"
            "Usuario: 'Hola' -> saludo\n"
            "Usuario: 'siguiente' -> paginacion\n"
            "Usuario: 'trabajos del autor Fernández' -> busqueda_por_autor\n"
            "Usuario: 'estudios sobre deserción' -> busqueda_por_tema\n"
            "Usuario: 'documentos de Bravo sobre mercado laboral' -> busqueda_por_autor\n"
            "Analiza la pregunta y responde SÓLO con la categoría.\n\n"
            "Pregunta: '{query_str}'\nCategoría:"
        )
        prompt = intent_classifier_prompt.format(query_str=user_message)
        response = Settings.llm.complete(prompt)
        intent = str(response).strip().lower().replace(" ", "_")
        print(f"Intención detectada por la IA: '{intent}'")
        return intent

    def extract_author_name(user_message):
        name_extractor_prompt = PromptTemplate(
            "Extrae el nombre de autor de la pregunta. Responde SÓLO con el nombre.\n"
            "Ejemplo 1: 'trabajos del autor Bravo' -> Bravo\n"
            "Ejemplo 2: 'documentos de Carolina Fernández sobre currículo' -> Carolina Fernández\n"
            "---\nPregunta: '{query_str}'\nNombre del autor:"
        )
        prompt = name_extractor_prompt.format(query_str=user_message)
        response = Settings.llm.complete(prompt)
        author_name = str(response).strip()
        print(f"Nombre de autor extraído por la IA: '{author_name}'")
        return author_name

    def handle_chitchat(user_message):
        chitchat_prompt = PromptTemplate(
            "Tu nombre es CEMIA, un asistente IA profesional y amable del Centro de Estudios del MINEDUC. "
            "Responde brevemente a la pregunta conversacional y redirige a tu función principal de buscar documentos.\n"
            "Ejemplo:\nUsuario: 'gracias'\nRespuesta: ¡De nada! Me alegra poder ayudar. ¿Hay algo más que necesites buscar?\n"
            "---\nPregunta: '{chitchat_query}'\nRespuesta:"
        )
        prompt = chitchat_prompt.format(chitchat_query=user_message)
        response = Settings.llm.complete(prompt)
        return str(response).strip()

    # --- FUNCIÓN PRINCIPAL CON ROUTER DE IA AVANZADO ---
    def unified_search(message, history):
        print("-" * 50); print(f"Recibida consulta: '{message}'")
        
        # Reiniciar estado para nuevas búsquedas, pero no para paginación o charla
        if not any(keyword in message.lower() for keyword in ['siguiente', 'más', 'pagina', 'página']):
             if get_user_intent(message) not in ['chitchat', 'saludo', 'ayuda']:
                chat_state.clear()
                chat_state.update({"last_query_term": None, "last_results": [], "current_page": 0})
        
        intent = get_user_intent(message)

        if intent == "saludo":
            return "¡Hola! Soy CEMIA, tu asistente de IA para el Centro de Estudios del MINEDUC. Puedes preguntarme por temas o autores. ¿En qué puedo asistirte hoy?"

        elif intent == "ayuda":
            return ("¡Claro! Soy CEMIA y te ayudo a buscar documentos.\n"
                    "**1. Búsqueda por Tema:** `Estudios sobre retención en primer año`\n"
                    "**2. Búsqueda por Autor:** `trabajos del autor Fernández`\n"
                    "**3. Búsqueda Combinada:** `documentos de Bravo sobre mercado laboral` (priorizará al autor)\n"
                    "**4. Navegación:** `siguiente` o `página 3` para ver más resultados.\n"
                    "¿Qué te gustaría buscar?")

        elif intent == "chitchat":
            return handle_chitchat(message)

        elif intent == "paginacion":
            if not chat_state["last_results"]: return "Primero necesitas hacer una búsqueda para poder ver más resultados."
            page_match = re.search(r"p[aá]gina\s+(\d+)", message.strip().lower())
            chat_state["current_page"] = int(page_match.group(1)) if page_match else chat_state["current_page"] + 1
            return format_results_page()
        
        elif intent == "busqueda_por_autor":
            author_name = extract_author_name(message)
            return search_by_author(author_name)

        else: # 'busqueda_por_tema' o por defecto
            return search_by_topic(message)

    # --- Creación de la Interfaz de Gradio ---
    chat_interface = gr.ChatInterface(
        fn=unified_search,
        chatbot=gr.Chatbot(height=500, type="messages"),
        type="messages",
        title="CEMIA (Asistente IA del Centro de Estudios)",
        description="Busca por tema o autor. El chatbot recordará tu última búsqueda para que puedas pedirle 'siguiente' o 'página 2'.",
        examples=["¿qué puedes hacer?", "estudios sobre brecha de género", "trabajos de Bravo", "siguiente"],
        theme="default",
    )

    print("\nLanzando la interfaz de CEMIA...")
    chat_interface.launch(inline=True, share=False)

Se identificaron 1887 autores únicos para la búsqueda precisa.

Lanzando la interfaz de CEMIA...
* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


--------------------------------------------------
Recibida consulta: '¿qué puedes hacer?'
Intención detectada por la IA: 'ayuda'
Intención detectada por la IA: 'ayuda'
--------------------------------------------------
Recibida consulta: 'hola'
Intención detectada por la IA: 'saludo'
Intención detectada por la IA: 'saludo'
--------------------------------------------------
Recibida consulta: 'estudios sobre brecha de género'
Intención detectada por la IA: 'busqueda_por_tema'
Intención detectada por la IA: 'busqueda_por_tema'
Búsqueda semántica por tema: 'estudios sobre brecha de género'.
Extrayendo concepto clave con LLM...
Término de búsqueda final: 'brecha de género, desigualdad de género, estudios de género, discriminación de género, participación femenina, representación femenina, techo de cristal'
